**Para importar y utilizar las funciones del módulo del proyecto, es necesario instalarlo previamente. Sigue las instrucciones detalladas en el archivo install.md, donde se describen los pasos necesarios para completar la instalación correctamente.**

In [1]:
%load_ext autoreload
%autoreload 2

In [89]:
import os
import re
# Importaciones específicas del proyecto
from energy_consumption_architecture.utils.paths import data_dir, data_raw_dir
from energy_consumption_architecture.clustering_utils import *
from energy_consumption_architecture.dataset import load_all_series
from energy_consumption_architecture.regresion_utils import pipeline_for_clusters

# Procesamiento de  datos 

**Antes de proceder a los pasos siguientes, el usuario debe realizar un preprocesamiento adecuado de las series de tiempo. Esto incluye tareas como la limpieza de datos, el tratamiento de valores nulos y el manejo de cualquier peculiaridad específica de las series de tiempo. Estas acciones son esenciales, ya que el análisis de series de tiempo puede variar significativamente según la naturaleza de los datos y el contexto en el que se utilicen.**

**Dado que las necesidades de preprocesamiento dependen del caso de uso, los objetivos y las características intrínsecas de las series de tiempo, este proceso no se estandarizará dentro de esta arquitectura. En su lugar, se deja al criterio del usuario la implementación de estas tareas, adaptándolas a las particularidades de cada conjunto de datos.** 

## Cargar datos 


Al clonar el proyecto, debe estar presente la carpeta `data`. Si esta carpeta no existe, será necesario crearla manualmente, ya que es fundamental para la organización de los datos del proyecto.

La función `data_dir` se utiliza para gestionar las rutas hacia la carpeta `data` y sus subdirectorios. 

- Si se llama sin argumentos (`data_dir()`), la función devuelve la ruta principal de la carpeta `data`.  
- Si se pasa un nombre como argumento, por ejemplo, `data_dir("raw")`, la función construye y devuelve la ruta al subdirectorio especificado dentro de `data`.

In [65]:
# Especifica el directorio donde están los archivos CSV
carpeta=data_dir("raw")
# Obtén la lista de todos los archivos en la carpeta
archivos = os.listdir(carpeta)
archivos[:5]

['RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Belleville-Scott.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Cahokia.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Carbondale-Southern.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_5A_USA_IL_CHICAGO-OHARE_Aurora.Muni.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_5A_USA_IL_CHICAGO-OHARE_Bloomington.csv']

La función `load_all_series` se utiliza para cargar múltiples series de tiempo desde archivos CSV y combinarlas en un único `DataFrame` para su análisis. 

- **Parámetros**: Recibe una lista con las rutas de los archivos y, opcionalmente, una lista de las columnas que se desean extraer. Si no se especifican las columnas, la función cargará todas las disponibles en cada archivo.
- **Validaciones**: Es importante asegurarse de que los archivos contengan las columnas especificadas, ya que de lo contrario se generará un error durante la carga.
- **Adiciones Automáticas**:
  - Asigna un identificador único (`ID`) a cada serie al momento de cargarla.
  - Extrae y almacena el nombre del archivo, útil si este describe información relevante sobre los datos.

In [66]:
# Define the columns to keep
columns_to_keep = [
    'Date/Time',
    'Cooling:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]

In [67]:
columns_to_keep = [
    'Date/Time',
    'Electricity:Facility [kW](Hourly)',
    'Fans:Electricity [kW](Hourly)',
    'Cooling:Electricity [kW](Hourly)',
    'Heating:Electricity [kW](Hourly)',
    'InteriorLights:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]

In [68]:
combined_df_filtered = load_all_series(archivos, columns_to_keep)

In [69]:
combined_df_filtered.head()

,Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,01/01 01:00:00,22.453919,3.998243,0.000733,0.0,4.589925,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,01/01 02:00:00,14.637149,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,01/01 03:00:00,14.651183,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,01/01 04:00:00,14.657947,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,01/01 05:00:00,14.806050,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


En este paso, es necesario asignar un nombre específico a la columna que contiene los *timestamps* de las series de tiempo. Se debe utilizar el nombre `"Date/Time"` para estandarizar y facilitar los pasos posteriores en el análisis.

Este proceso no se ha estandarizado completamente dentro de la arquitectura, ya que el tratamiento de las series de tiempo puede variar según las características del dataset. Por lo tanto, el usuario debe ajustar este paso para adaptarlo a su conjunto de datos, asegurándose de que la columna de *timestamps* sea convertida al tipo de dato `datetime`. Esto es fundamental para que las funciones posteriores trabajen correctamente con la serie temporal.

In [70]:
# Procesar la columna de fecha y tiempo
combined_df_filtered.rename(columns={"Date/Time": "Date/Time"}, inplace=True)
combined_df_filtered["Date/Time"] = '2004 ' + combined_df_filtered["Date/Time"]
date_format = '%Y %m/%d %H:%M:%S'
combined_df_filtered["Date/Time"] = pd.to_datetime(combined_df_filtered["Date/Time"], format=date_format, errors='coerce')

In [71]:
combined_df_filtered.head()

,Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,2004-01-01 01:00:00,22.453919,3.998243,0.000733,0.0,4.589925,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,2004-01-01 02:00:00,14.637149,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,2004-01-01 03:00:00,14.651183,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,2004-01-01 04:00:00,14.657947,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,2004-01-01 05:00:00,14.806050,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


## Obtener caracteristicas de las series de tiempo

Se extraen características representativas de las series temporales, específicamente la media y la desviación estándar de cada columna del DataFrame. Estas estadísticas resumen el comportamiento general de las series y proporcionan una base para el análisis y modelado posterior.

In [72]:
df_stats = calculate_statistics(combined_df_filtered)

In [73]:
df_stats.head()

,series_id,Electricity:Facility [kW](Hourly)_mean,Electricity:Facility [kW](Hourly)_std_dev,Fans:Electricity [kW](Hourly)_mean,Fans:Electricity [kW](Hourly)_std_dev,Cooling:Electricity [kW](Hourly)_mean,Cooling:Electricity [kW](Hourly)_std_dev,Heating:Electricity [kW](Hourly)_mean,Heating:Electricity [kW](Hourly)_std_dev,InteriorLights:Electricity [kW](Hourly)_mean,InteriorLights:Electricity [kW](Hourly)_std_dev,InteriorEquipment:Electricity [kW](Hourly)_mean,InteriorEquipment:Electricity [kW](Hourly)_std_dev
0,series_1,36.968830,13.139998,3.331869,1.490142,3.073887,6.226848,0.0,0.0,7.522378,2.960208,18.995908,7.265027
1,series_2,37.357872,13.459672,3.327967,1.488397,3.446951,6.596764,0.0,0.0,7.522378,2.960208,18.995908,7.265027
2,series_3,37.896555,13.945362,3.327124,1.488020,3.973240,7.146033,0.0,0.0,7.522378,2.960208,18.995908,7.265027
3,series_4,35.892184,12.083141,3.303988,1.477673,2.055950,4.688730,0.0,0.0,7.522378,2.960208,18.995908,7.265027
4,series_5,35.976038,12.127984,3.323298,1.486309,2.114192,4.797245,0.0,0.0,7.522378,2.960208,18.995908,7.265027


# Clustering

Esta función utiliza las características de las series de tiempo para evaluar cuál método de agrupación ofrece los mejores resultados. Se prueba la agrupación tanto con todas las características originales como con las reducidas mediante **PCA**. Se implementan tres algoritmos de clustering: **K-Means**, **DBSCAN**, y **Jerárquico**. Como salida, la función proporciona:

1. Una tabla con las métricas de evaluación para cada método.
2. El pipeline del proceso con el mejor rendimiento.
3. Los datos etiquetados con el grupo asignado a cada muestra.

In [74]:
# Ejecutar el pipeline automatizado
metrics, best_pipeline, clustered_data = automated_clustering_pipeline(df_stats)

C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\envs\energy_consumption_architecture\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\ASUS TUF\.conda\

In [75]:
# Resultados
print("Métricas de los métodos evaluados:")
metrics.sort_values(by="Combined Score",ascending=False)

Métricas de los métodos evaluados:


,Model,Silhouette Score,Davies-Bouldin Index,Num Clusters,PCA Applied,Silhouette Score Norm,Davies-Bouldin Index Norm,Combined Score
3,K-Means,0.761276,0.226323,4,True,1.000000,1.000000,1.000000
0,K-Means,0.743379,0.250698,4,False,0.937277,0.973770,0.955524
5,Hierarchical,0.710257,0.660865,2,True,0.821192,0.532385,0.676788
2,Hierarchical,0.699690,0.671114,2,False,0.784160,0.521355,0.652757
4,DBSCAN,0.518675,1.047642,9,True,0.149757,0.116170,0.132963
1,DBSCAN,0.475944,1.155595,9,False,0.000000,0.000000,0.000000


In [76]:
print("\nPipeline del mejor proceso:")
best_pipeline


Pipeline del mejor proceso:


Pipeline(steps=[('scaler', StandardScaler()),
                ('pca', PCA(n_components=np.int64(4))),
                ('clustering', KMeans(n_clusters=4, random_state=42))])

In [77]:
# Mostrar las primeras filas del DataFrame con clusters asignados
clustered_data.loc[:,["Cluster"]].value_counts()

Cluster
0          65
1           5
2           5
3           5
Name: count, dtype: int64

# Regresion

In [82]:
# Define the columns to keep
columns_to_keep = [
    'Date/Time',
    'Electricity:Facility [kW](Hourly)',
    'Fans:Electricity [kW](Hourly)',
    'Cooling:Electricity [kW](Hourly)',
    'Heating:Electricity [kW](Hourly)',
    'InteriorLights:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]
data_complete = load_all_series(archivos, columns_to_keep)
# Procesar la columna de fecha y tiempo
data_complete.rename(columns={"Date/Time": "Date/Time"}, inplace=True)
data_complete["Date/Time"] = '2004 ' + data_complete["Date/Time"]
date_format = '%Y %m/%d %H:%M:%S'
data_complete["Date/Time"] = pd.to_datetime(data_complete["Date/Time"], format=date_format, errors='coerce')

In [83]:
data_complete.head()

,Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,2004-01-01 01:00:00,22.453919,3.998243,0.000733,0.0,4.589925,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,2004-01-01 02:00:00,14.637149,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,2004-01-01 03:00:00,14.651183,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,2004-01-01 04:00:00,14.657947,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,2004-01-01 05:00:00,14.806050,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


In [84]:
# Asegúrate de que el DataFrame `df_stats` contenga las etiquetas de cluster y el `series_id`
# Y que el DataFrame `data_complete` tenga el `series_id`

average_time_series_by_cluster = calculate_average_time_series_by_cluster(data_complete, clustered_data)

In [85]:
average_time_series_by_cluster.head()

,Cluster,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly)
Date/Time,,,,,,,
2004-01-01 01:00:00,0,47.210969,5.789775,3.010854,4.637630,6.918895,10.669373
2004-01-01 02:00:00,0,47.047291,5.946731,3.048947,4.450657,6.588343,10.525933
2004-01-01 03:00:00,0,45.619393,5.862989,2.883068,5.406697,5.084587,10.308935
2004-01-01 04:00:00,0,45.697195,6.186896,2.968095,4.801340,5.084587,10.297614
2004-01-01 05:00:00,0,47.030928,5.832209,2.942949,5.603789,5.182567,10.482863


In [86]:
target = 'Electricity:Facility [kW](Hourly)'

In [90]:
# Cargar modelos con configuraciones ajustadas
models = {
    "Linear Regression": LinearRegression(fit_intercept=True, n_jobs=-1),
    "Tree": DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42),
    "SVM": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=10, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, subsample=0.8, colsample_bytree=0.8, random_state=42)
}

NameError: name 'LinearRegression' is not defined

In [88]:
metrics_df, best_models_df = pipeline_for_clusters(average_time_series_by_cluster, target, models,threshold_ratio=2)

NameError: name 'models' is not defined

In [61]:
best_models_df

,Model,Train RMSE,Train MAE,Train R2,Test RMSE,Test MAE,Test R2,Model Name
4,"XGBRegressor(base_score=None, booster=None, ca...",1.677753,1.242559,0.996641,2.418129,1.875946,0.990515,XGBoost
0,LinearRegression(n_jobs=-1),34.509758,27.981590,0.994399,39.835457,31.613485,0.988051,Linear Regression
4,"XGBRegressor(base_score=None, booster=None, ca...",13.238075,9.829072,0.996658,24.246907,19.310649,0.990215,XGBoost
0,LinearRegression(n_jobs=-1),6.566537,5.603544,0.999608,6.617337,5.642924,0.998601,Linear Regression
